# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770850263317                   -0.52    9.0         
  2   -2.772141238150       -2.89       -1.32    1.0    144ms
  3   -2.772170353467       -4.54       -2.52    1.0    148ms
  4   -2.772170679598       -6.49       -3.41    1.0    177ms
  5   -2.772170720859       -7.38       -3.85    2.0    169ms
  6   -2.772170722911       -8.69       -4.79    1.0    171ms
  7   -2.772170723011      -10.00       -5.22    2.0    172ms
  8   -2.772170723015      -11.37       -6.16    1.0    167ms
  9   -2.772170723015      -13.39       -6.36    2.0    179ms
 10   -2.772170723015      -13.51       -7.46    1.0    188ms
 11   -2.772170723015      -14.18       -8.13    2.0    191ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770750583513                   -0.53    9.0         
  2   -2.7

1.773557968479249

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770845390422                   -0.52    9.0         
  2   -2.772062853989       -2.91       -1.32    1.0    197ms
  3   -2.772083127049       -4.69       -2.48    1.0    147ms
  4   -2.772083353743       -6.64       -3.19    1.0    148ms
  5   -2.772083417021       -7.20       -4.04    2.0    225ms
  6   -2.772083417734       -9.15       -4.76    1.0    151ms
  7   -2.772083417810      -10.12       -5.80    2.0    192ms
  8   -2.772083417810      -12.27       -6.03    2.0    177ms
  9   -2.772083417811      -12.81       -7.16    1.0    185ms
 10   -2.772083417811   +    -Inf       -7.65    2.0    185ms
 11   -2.772083417811   +  -14.31       -8.23    1.0    170ms

Polarizability via ForwardDiff:       1.7725349621726858
Polarizability via finite difference: 1.773557968479249
